## PART 1: Get data
## Importing Pandas

## Use pandas to read directly wiki html table

In [1]:
import pandas as pd
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)[0]

## Check dataframe

In [2]:
dfs.head()
dfs = dfs.rename(columns={'Neighbourhood': 'Neighborhood'})

## Ignore cells with a borough that is __Not assigned__

In [3]:
dfs = dfs[dfs['Borough']!='Not assigned']

## Combine Neighbourhood with same postcode

In [4]:
dfs = dfs.groupby(['Postcode','Borough'])['Neighborhood'].apply(lambda x:', '.join(x)).reset_index()

## Check cell that has a borough but a __Not assigned__ neighborhood

In [5]:
try:
    condition = dfs['Neighborhood'] == 'Not assigned'
    print("'Not assigned' neighborhood is ", dfs[condition]['Borough'].values[0])
except IndexError:
    print("'Not assigned' neighborhood not found")

'Not assigned' neighborhood is  Queen's Park


## Assign neighbourhood value same as borough

In [6]:
dfs['Neighborhood'].loc[condition] = dfs[condition]['Borough'].values[0]

## Assign neighbourhood value same as borough

In [7]:
try:
    dfs['Neighborhood'].unique()['Not assigned']
except IndexError:
    print("'Not assigned' neighborhood not found")

'Not assigned' neighborhood not found


## Check dataframe

In [8]:
dfs.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Check dataframe shape

In [9]:
dfs.shape

(103, 3)

## End of Part 1

## PART 2:
## Function to extract lat long using arcgis api

In [10]:
#!conda install -c conda-forge geocoder --yes # uncomment this line if you haven't completed the Foursquare API lab

In [11]:
# just incase
#!wget -q -O 'toronto.csv' https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
#coords_dl = pd.read_csv('toronto.csv')

In [12]:
import geocoder # import geocoder

def get_latlng(pcode):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      ag = geocoder.arcgis(f'{pcode}, Canada')
      lat_lng_coords = ag.latlng
    
    return lat_lng_coords

## get values and put into lists

In [13]:
lat = []
lng = []
for pcode in dfs['Postcode'].values:
    cord = get_latlng(pcode)
    lat.append(cord[0])
    lng.append(cord[1])

lat[:5],lng[:5]

([43.811525000000074,
  43.78566500000005,
  43.76581500000003,
  43.768369121000035,
  43.76968799900004],
 [-79.19551746399998,
  -79.15872457299997,
  -79.17519294699997,
  -79.21758999999997,
  -79.23943999999995])

## Assign list to dataframe columns

In [14]:
dfs['Latitude'] = lat
dfs['Longitude'] = lng
dfs.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


## Check Dataframe Shape

In [15]:
dfs.shape

(103, 5)

## End of Part 2

## PART 3: Explore and cluster the neighborhoods in Toronto

In [16]:
#!conda install -c conda-forge geopy --yes
#!conda install -c conda-forge folium --yes 

In [17]:
from geopy.geocoders import Nominatim
import folium

address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(dfs['Latitude'], dfs['Longitude'], dfs['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [19]:
import requests

In [20]:
CLIENT_ID = '5MWD2HZZ3MKR5RMCG3WNCXM3AGIZCW242RENA1AAGS3YPKDJ' # your Foursquare ID
CLIENT_SECRET = 'PX4AKHKLIGQ5EZAABH4SZGLKM31H2UTFMMQKQLOQM512P5U3' # your Foursquare Secret
VERSION = '20200203' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5MWD2HZZ3MKR5RMCG3WNCXM3AGIZCW242RENA1AAGS3YPKDJ
CLIENT_SECRET:PX4AKHKLIGQ5EZAABH4SZGLKM31H2UTFMMQKQLOQM512P5U3


## Explore Neighborhoods in Toronto

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=850):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get nearby venues of Toronto

In [22]:
toronto_venues = getNearbyVenues(names=dfs['Neighborhood'],
                                   latitudes=dfs['Latitude'],
                                   longitudes=dfs['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [23]:
print(toronto_venues.shape)
toronto_venues.head()

(4240, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.811525,-79.195517,Canadiana exhibit,43.817962,-79.193374,Zoo Exhibit
1,"Rouge, Malvern",43.811525,-79.195517,Wendy's,43.807448,-79.199056,Fast Food Restaurant
2,"Rouge, Malvern",43.811525,-79.195517,Ontrack Services,43.816399,-79.199157,Financial or Legal Service
3,"Rouge, Malvern",43.811525,-79.195517,Grizzly Bear Exhibit,43.817031,-79.193458,Zoo Exhibit
4,"Rouge, Malvern",43.811525,-79.195517,Upper Rouge Trail,43.809988,-79.186147,Trail


### Check Dataframe

In [24]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,40,40,40,40,40,40
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",7,7,7,7,7,7
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",15,15,15,15,15,15
"Alderwood, Long Branch",18,18,18,18,18,18
"Bathurst Manor, Downsview North, Wilson Heights",26,26,26,26,26,26
Bayview Village,6,6,6,6,6,6
"Bedford Park, Lawrence Manor East",32,32,32,32,32,32
Berczy Park,100,100,100,100,100,100


### Check unique categories

In [25]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 329 uniques categories.


### One hot encoding for each categories

In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
toronto_onehot.shape

(4240, 329)

### Group rows by getting the mean of the frequency of occurrence of each category

In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Animal Shelter,Antique Shop,Arcade,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.00,0.00,0.0,0.000,0.020000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00,0.0,0.010000,0.00,0.000000,0.000000,0.000000
1,Agincourt,0.0,0.00,0.00,0.0,0.000,0.025000,0.0,0.000000,0.0,...,0.000000,0.000000,0.025000,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.00,0.00,0.0,0.000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.00,0.00,0.0,0.000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.000000
4,"Alderwood, Long Branch",0.0,0.00,0.00,0.0,0.000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.00,0.00,0.0,0.000,0.000000,0.0,0.000000,0.0,...,0.000000,0.038462,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.000000
6,Bayview Village,0.0,0.00,0.00,0.0,0.000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.000000
7,"Bedford Park, Lawrence Manor East",0.0,0.00,0.00,0.0,0.000,0.031250,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.000000
8,Berczy Park,0.0,0.00,0.00,0.0,0.000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.000000
9,"Birch Cliff, Cliffside West",0.0,0.00,0.00,0.0,0.000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.000000


### Check dataframe

In [29]:
toronto_grouped.shape

(101, 329)

In [30]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.07
2        Hotel  0.06
3   Steakhouse  0.04
4   Restaurant  0.04


----Agincourt----
                venue  freq
0  Chinese Restaurant  0.18
1       Shopping Mall  0.08
2         Pizza Place  0.05
3    Department Store  0.02
4       Grocery Store  0.02


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                  venue  freq
0                  Park  0.29
1              Pharmacy  0.14
2        Shop & Service  0.14
3                   Gym  0.14
4  Caribbean Restaurant  0.14


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
           venue  freq
0  Grocery Store  0.13
1    Coffee Shop  0.13
2       Pharmacy  0.07
3   Liquor Store  0.07
4       Gym Pool  0.07


----Alderwood, Long Branch----
          venue  freq
0      Pharmacy  0.11
1   Pizza Place  0.11
2          Pool  0.06
3  Liquor Store  0.06
4    

                  venue  freq
0        Clothing Store  0.12
1  Fast Food Restaurant  0.08
2           Coffee Shop  0.07
3   Japanese Restaurant  0.04
4            Shoe Store  0.04


----First Canadian Place, Underground city----
                venue  freq
0         Coffee Shop  0.09
1               Hotel  0.07
2                Café  0.07
3          Steakhouse  0.04
4  Italian Restaurant  0.03


----Flemingdon Park, Don Mills South----
                 venue  freq
0       Science Museum  0.07
1  Japanese Restaurant  0.07
2           Beer Store  0.07
3          Coffee Shop  0.07
4                  Gym  0.07


----Forest Hill North, Forest Hill West----
              venue  freq
0              Café  0.08
1  Sushi Restaurant  0.08
2              Park  0.08
3          Bus Line  0.04
4             Trail  0.04


----Glencairn----
                venue  freq
0       Grocery Store  0.12
1         Pizza Place  0.08
2         Gas Station  0.08
3  Italian Restaurant  0.08
4      Sandwich Place  0

         venue  freq
0  Coffee Shop  0.09
1        Hotel  0.05
2         Café  0.05
3   Steakhouse  0.04
4          Bar  0.03


----Studio District----
                venue  freq
0         Coffee Shop  0.07
1                Café  0.05
2                 Bar  0.04
3              Bakery  0.04
4  Italian Restaurant  0.04


----The Annex, North Midtown, Yorkville----
                venue  freq
0  Italian Restaurant  0.07
1                 Pub  0.07
2         Coffee Shop  0.06
3      Sandwich Place  0.06
4                Café  0.06


----The Beaches----
                 venue  freq
0          Coffee Shop  0.07
1                  Pub  0.07
2  Japanese Restaurant  0.04
3          Pizza Place  0.04
4            Gastropub  0.04


----The Beaches West, India Bazaar----
               venue  freq
0  Indian Restaurant  0.08
1        Coffee Shop  0.05
2               Café  0.05
3      Grocery Store  0.05
4        Pizza Place  0.03


----The Danforth West, Riverdale----
              venue  freq
0 

### Find Most Common Venues here I choose only 5

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
import numpy as np

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Steakhouse,Restaurant
1,Agincourt,Chinese Restaurant,Shopping Mall,Pizza Place,Lounge,Sandwich Place
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Caribbean Restaurant,Shop & Service,Pizza Place,Pharmacy
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Coffee Shop,Hardware Store,Pharmacy,Fried Chicken Joint
4,"Alderwood, Long Branch",Pizza Place,Pharmacy,Convenience Store,Gym,Sandwich Place


## Cluster Neighborhoods

In [33]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 0], dtype=int32)

In [34]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Steakhouse,Restaurant
1,Agincourt,Chinese Restaurant,Shopping Mall,Pizza Place,Lounge,Sandwich Place
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Caribbean Restaurant,Shop & Service,Pizza Place,Pharmacy
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Coffee Shop,Hardware Store,Pharmacy,Fried Chicken Joint
4,"Alderwood, Long Branch",Pizza Place,Pharmacy,Convenience Store,Gym,Sandwich Place


In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dfs

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(inplace=True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype('int')
toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517,2,Zoo Exhibit,Financial or Legal Service,Trail,Fast Food Restaurant,Event Space
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725,0,Bar,Park,Breakfast Spot,Burger Joint,Cupcake Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193,0,Park,Gymnastics Gym,Gym / Fitness Center,Convenience Store,Pizza Place
3,M1G,Scarborough,Woburn,43.768369,-79.217590,1,Sandwich Place,Fast Food Restaurant,Supermarket,Coffee Shop,Park
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440,1,Bakery,Indian Restaurant,Hakka Restaurant,Caribbean Restaurant,Chinese Restaurant


### Visualize the clustering

In [36]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### List out first cluster; apparently consists of Parks and Leisure locations

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Scarborough,0,Bar,Park,Breakfast Spot,Burger Joint,Cupcake Shop
2,Scarborough,0,Park,Gymnastics Gym,Gym / Fitness Center,Convenience Store,Pizza Place
9,Scarborough,0,Park,Hotel,Café,College Stadium,Skating Rink
14,Scarborough,0,Park,Caribbean Restaurant,Shop & Service,Pizza Place,Pharmacy
17,North York,0,Park,Bakery,Pharmacy,Residential Building (Apartment / Condo),Coffee Shop
20,North York,0,Cafeteria,Pool,Martial Arts Dojo,Park,Farm
25,North York,0,Park,Bus Stop,Skating Rink,Furniture / Home Store,Shop & Service
32,North York,0,Construction & Landscaping,Health & Beauty Service,Park,Baseball Field,Home Service
48,Central Toronto,0,Park,Grocery Store,Playground,Gym / Fitness Center,Café
50,Downtown Toronto,0,Park,Playground,Grocery Store,Candy Store,Trail


### List out second cluster; apparently consists of dining locations

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Scarborough,1,Sandwich Place,Fast Food Restaurant,Supermarket,Coffee Shop,Park
4,Scarborough,1,Bakery,Indian Restaurant,Hakka Restaurant,Caribbean Restaurant,Chinese Restaurant
5,Scarborough,1,Sandwich Place,Ice Cream Shop,Fast Food Restaurant,Convenience Store,Indian Restaurant
6,Scarborough,1,Coffee Shop,Chinese Restaurant,Light Rail Station,Grocery Store,Department Store
7,Scarborough,1,Diner,Intersection,Bakery,Bus Line,Convenience Store
8,Scarborough,1,Ice Cream Shop,Burger Joint,Park,Sandwich Place,Bank
10,Scarborough,1,Bakery,Pet Store,Vietnamese Restaurant,Indian Restaurant,Electronics Store
11,Scarborough,1,Middle Eastern Restaurant,Pizza Place,Grocery Store,Burger Joint,Indian Restaurant
12,Scarborough,1,Chinese Restaurant,Shopping Mall,Pizza Place,Lounge,Sandwich Place
13,Scarborough,1,Fast Food Restaurant,Bakery,Coffee Shop,Vietnamese Restaurant,Shopping Mall


### List out third cluster; apparently also consists of Other

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Scarborough,2,Zoo Exhibit,Financial or Legal Service,Trail,Fast Food Restaurant,Event Space
